In [3]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
% matplotlib inline

In [4]:
#Tests to see if there is a collision between two pool balls
#Input: Position vectors, velocity vectors, timestep dt
#Output: True or False
R = 2
def collision_detection_bb(r1,r2,v1,v2,dt):
    r1_f = r1 + v1*dt
    r2_f = r2 + v2*dt
    
    r_f = np.abs(r1_f - r2_f)
    r = np.sqrt(np.dot(r_f,r_f))
    
    if r < 2*R:
        return True
    else:
        return False


#Tests to see if there is a pool ball and wall collision
#Input: Position vectors, velocity vectors, timestep dt
#Output: True or False
def collision_detection_bw(r1,v1,dt):
    Table_x = 4
    Table_y = 10
    TABLE_DIM = np.array([4,10])
    r1_f = r1 + v1*dt
    
    
    if r1_f[0] < R:
        return True
    elif r1_f[1] < R:
        return True
    elif Table_x - r1_f[0] < R:
        return True
    elif Table_y - r1_f[1] < R:
        return True
    else:
        return False


In [1]:
def BallOnBallCollision(x1,v1,w1,x2,v2,w2,timeStep,Rad,m,mu):
    
    #x 1 and 2 are entire position vectors, not horizontal location vectors
    a = (np.dot(v1,v1)-2*np.dot(v1,v2)+np.dot(v2,v2))
    b = (2*np.dot(v1,x1)-2*np.dot(v1,x2)-2*np.dot(v2,x1)+2*np.dot(v2,x2))
    c = (np.dot(x1,x1)-2*np.dot(x1,x2)+np.dot(x2,x2)-(2*Rad)**2)
    
    #time after beginning of timestep
    StrikeTime = (- b - np.sqrt(b**2 - 4*a*c))/(2*a)
    
    #time left in timestep
    RemainingTimeInStep = timeStep-StrikeTime
    
    #center of ball locations at collision
    Strikex1 = x1 + v1*StrikeTime
    Strikex2 = x2 + v2*StrikeTime
    
    #unit vector normal to collision plane
    n = (Strikex1-Strikex2)/np.sqrt(np.dot((Strikex1-Strikex2),(Strikex1-Strikex2)))
   
    #normal component of velocities to collision plane
    v1normal = np.dot(v1,-n)*(-n)
    v2normal = np.dot(v2,n)*(n)
    
    #tangential component of velocities to collision plane
    v1tang = v1 - v1normal
    v2tang = v2 - v2normal
    
    #outgoing linear velocities after collision
    Strikev1 = v1tang + v2normal
    Strikev2 = v2tang + v1normal
    
    #radial vector from ball center to strike point
    StrikeR1 = Rad*(-n)
    StrikeR2 = Rad*n
    
    #perimiter velocity of ball at strike point
    vPerim1 = np.cross(StrikeR1,w1)
    vPerim2 = np.cross(StrikeR2,w2)
    print vPerim1
    print vPerim2
    #relative perimiter velocity for ball 1 and ball 2
    vPerimRel1 = vPerim1 - vPerim2
    vPerimRel2 = vPerim2 - vPerim1
    
    #change in normal velocity
    delVNormal1 = np.sqrt(np.dot(v2normal - v1normal,v2normal - v1normal))
    delVNormal2 = np.sqrt(np.dot(v1normal - v2normal, v1normal - v2normal))
    #change in angular momentum
    #needs fixing
    delW1 = (delVNormal2)*(-5*m*mu/(2*m*(Rad**2)))*np.cross(StrikeR1, 
            (vPerimRel1 + v1tang - v2tang)/np.absolute(np.sqrt(np.dot(vPerimRel1 + 
            v1tang - v2tang,vPerimRel1 + v1tang - v2tang))))
    
    delW2 = (delVNormal1)*(-5*m*mu/(2*m*(Rad**2)))*np.cross(StrikeR2, 
            (vPerimRel2 + v2tang - v1tang)/np.absolute(np.sqrt(np.dot(vPerimRel2 + 
            v2tang - v1tang,vPerimRel2 + v2tang - v1tang))))
    
    newW1 = w1 + delW1
    newW2 = w2 + delW2
    return [StrikeTime,Strikex1,Strikex2,Strikev1,Strikev2,newW1,newW2]

def EnergyTest(v1i,v2i,w1i,w2i,v1f,v2f,w1f,w2f,m,r):
    
    KEI1 = .5*m*D3VectorAbs(v1i)**2 + .2*(m*r**2)*D3VectorAbs(w1i)**2
    KEI2 = .5*m*D3VectorAbs(v2i)**2 + .2*(m*r**2)*D3VectorAbs(w2i)**2
    KEF1 = .5*m*D3VectorAbs(v1f)**2 + .2*(m*r**2)*D3VectorAbs(w1f)**2
    KEF2 = .5*m*D3VectorAbs(v2f)**2 + .2*(m*r**2)*D3VectorAbs(w2f)**2
    
    DelKE = KEI1 + KEI2 - KEF1 - KEF2
    
    if DelKE > .1:
        print("ya done fucked up, son")
    
    if DelKE < -.1:
        print("ya done fucked up, son")
    else:
        print("hell yea")
    
def D3VectorAbs(v):
    
    return np.sqrt(v[0]**2 + v[1]**2 + v[2]**2)



In [5]:
d1 = np.asarray([1.5,0,0])
d2 = np.asarray([3,0,0])
v1 = np.asarray([1,.2,0])
v2 = np.asarray([0,0,0])
w1 = np.asarray([0,0,1])
w2 = np.asarray([0,0,1])
dT = 1
Rad = .5
m = 1
mu = .1
colData = BallOnBallCollision(d1,v1,w1,d2,v2,w2,dT,Rad,m,mu)
print colData
EnergyTest(v1,v2,w1,w2,colData[1],colData[3],colData[2],colData[4],m,Rad)

[-0.05051159 -0.49744204  0.        ]
[ 0.05051159  0.49744204  0.        ]
[0.50511592838375352, array([ 2.00511593,  0.10102319,  0.        ]), array([ 3.,  0.,  0.]), array([ 0.03030696,  0.29846522,  0.        ]), array([ 0.96969304, -0.09846522,  0.        ]), array([ 0.        ,  0.        ,  1.48733972]), array([ 0.        ,  0.        ,  1.48733972])]
ya done fucked up, son
